# ***##step by step denoise##***

In [ ]:
from PIL import Image
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler, ControlNetModel
import os
import numpy as np

from diffusers import StableDiffusionControlNetPipeline, StableDiffusionControlNetInpaintPipeline, UniPCMultistepScheduler, DDIMScheduler
from diffusers.utils import load_image
from torchvision import transforms
import imgviz

import cv2
from matplotlib import pyplot as plt
import PIL

In [ ]:
prompt = ["crack"]
device = torch_device= 'cuda'
seed = 1
height = 512  # default height of Stable Diffusion
width = 512  # default width of Stable Diffusion
num_inference_steps = 25  # Number of denoising steps
guidance_scale = 7.5  # Scale for classifier-free guidance
generator = torch.Generator(device=device).manual_seed(seed)  # Seed generator to create the initial latent noise
batch_size = len(prompt)

## 0.图像准备，原图，image_mask

### 0.1.检测和定位损伤

In [ ]:
# img 想增加损伤的图片
img_dir = '/home/ubunto/Project/konglx/generate/ControlNet-v1-1-nightly/training/origin_dataset/open_datasets_element_cls_open/JPEGImages/cls_a1206.jpg'
image_org = load_image(img_dir)
print(image_org.mode)
image_org

In [ ]:
# img 检测的segmentation 选择添加损伤的构件
img_seg_dir = '/home/ubunto/Project/konglx/generate/ControlNet-v1-1-nightly/training/origin_dataset/open_datasets_element_cls_open/SegmentationClass/cls_a1206.png'
image_seg = Image.open(img_seg_dir)
print(image_seg.mode)
image_seg

In [ ]:
def colored_mask(mask, save_path=None):
    lbl_pil = Image.fromarray(mask.astype(np.uint8), mode="P")
    colormap = imgviz.label_colormap()
    # print(colormap)
    lbl_pil.putpalette(colormap.flatten())
    if save_path is not None:
        lbl_pil.save(save_path)

    return lbl_pil

In [ ]:
# 提取对应部位的mask单体
image_seg_np = np.array(image_seg)
image_seg_labels = np.unique(image_seg_np)

list_labels = []
for label in image_seg_labels:
    if label == 0:
        continue
    label_zero = np.zeros_like(image_seg_np)
    label_zero[image_seg_np == label] = label
    list_labels.append(label_zero)
print(len(list_labels))

labels_unique_list = []
for i in list_labels:
    labels_unique_list.append(int(np.unique(i)[1]))
print(labels_unique_list)

print(labels_unique_list.index(6))
label_np = list_labels[labels_unique_list.index(6)]
label_type = int(np.unique(label_np)[1])
print(label_type,type(label_type))
print(type(label_np), label_np.shape, np.unique(label_np))
label_pil = colored_mask(label_np)
print(label_np[200][200])
label_pil

In [ ]:
def maximum_internal_rectangle_input_np(mask_np):
    # img = cv2.imread(mask_path)
    # img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # ret, img_bin = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY)
    mask_type = int(np.unique(mask_np)[1])
 
    contours, _ = cv2.findContours(mask_np, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    # print(len(contours))
 
    contour = contours[0].reshape(len(contours[0]), 2)
    # print(contour,len(contour))
 
    rect = []
 
    for i in range(len(contour)):
        x1, y1 = contour[i]
        for j in range(len(contour)):
            x2, y2 = contour[j]
            area = abs(y2 - y1) * abs(x2 - x1)
            rect.append(((x1, y1), (x2, y2), area))
 
    all_rect = sorted(rect, key=lambda x: x[2], reverse=True)
 
    if all_rect:
        best_rect_found = False
        index_rect = 0
        nb_rect = len(all_rect)
 
        while not best_rect_found and index_rect < nb_rect:
 
            rect = all_rect[index_rect]
            (x1, y1) = rect[0]
            (x2, y2) = rect[1]
 
            valid_rect = True
 
            x = min(x1, x2)
            while x < max(x1, x2) + 1 and valid_rect:
                if mask_np[y1][x] == 0 or mask_np[y2][x] == 0:
                    valid_rect = False
                x += 1
 
            y = min(y1, y2)
            while y < max(y1, y2) + 1 and valid_rect:
                if mask_np[y][x1] == 0 or mask_np[y][x2] == 0:
                    valid_rect = False
                y += 1
 
            if valid_rect:
                best_rect_found = True
 
            index_rect += 1
 
        if best_rect_found:
            # 如果要在灰度图img_gray上画矩形，请用黑色画（0,0,0）
            cv2.rectangle(mask_np, (x1, y1), (x2, y2), (0, 0, 0), 2)
            # cv2.imshow("rec", mask_np)
            # cv2.waitKey(0)
            plt.imshow(mask_np)
            list_xy = list([(x1, y1), (x2, y2)])
            return mask_np, list_xy
 
        else:
            print("No rectangle fitting into the area")
 
    else:
        print("No rectangle found")

In [ ]:
mask_np_max, xy_list = maximum_internal_rectangle_input_np(label_np)

In [ ]:
# 座标点和尺寸面积, h_mask, w_mask
w_mask_org = abs(xy_list[0][0]-xy_list[1][0])
w_mask = w_mask_org if w_mask_org % 8 ==0 else 8*(w_mask_org//8)

h_mask_org = abs(xy_list[0][1]-xy_list[1][1])
h_mask = h_mask_org if h_mask_org % 8 ==0 else 8*(h_mask_org//8)

xy_list, abs((xy_list[0][0]-xy_list[1][0])*(xy_list[0][1]-xy_list[1][1])), h_mask, w_mask

In [ ]:
# resize裂缝区域为1
mask_one = np.ones((h_mask, w_mask), dtype=np.uint8)*255
# resize_crack_mask_np = np.array(mask_copy_cvt_l.resize((abs(xy_list[0][0]-xy_list[1][0]), abs(xy_list[0][1]-xy_list[1][1])), Image.NEAREST))
print(np.unique(mask_one))
mask_one_pil = Image.fromarray(mask_one, mode='L')
mask_one_pil

In [ ]:
# 先创建zero的mask与原裂缝的mask尺寸一致，再将裂缝的mask对应到xy_list的座标点位置，

new_crack_mask_np = np.zeros_like(label_np)

mask_one_pil_zero = Image.fromarray(new_crack_mask_np)
mask_one_pil_zero.paste(mask_one_pil, (xy_list[0][0], xy_list[1][1]))
mask_one_pt_l = transforms.ToTensor()(mask_one_pil_zero)
mask_one_pt_l_4dim = mask_one_pt_l.unsqueeze(0)
print(mask_one_pt_l_4dim.shape, mask_one_pt_l_4dim.min(), mask_one_pt_l_4dim.max())  ###################################### mask_one_pt_l_4dim  #########################

mask_one_pil_zero_rgb = mask_one_pil_zero.convert('RGB')
mask_one_zero_rgb_pt = transforms.ToTensor()(mask_one_pil_zero_rgb)
print(mask_one_zero_rgb_pt.shape) 
mask_only_mask_pt_4dim = mask_one_zero_rgb_pt.unsqueeze(0) 
print(mask_only_mask_pt_4dim.shape)   ###################################### mask_only_mask_pt_4dim  #########################（c=3）先不用
mask_one_pil_np_paste = np.array(mask_one_pil_zero)
print(mask_one_pil_np_paste.shape, np.unique(mask_one_pil_np_paste))
mask_one_pil_rgb_np_paste = np.array(mask_one_pil_zero_rgb)
print(mask_one_pil_rgb_np_paste.shape, np.unique(mask_one_pil_rgb_np_paste))
print(mask_one_pil_zero.mode)
mask_one_pil_zero

In [ ]:
from torchvision import transforms

mask_dir = '/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/crack（复件）/SegmentationClass/DeepCrack_11240-6.png'
validation_image = Image.open(mask_dir).convert("RGB")
validation_image.size

conditioning_image_transforms = transforms.Compose(
        [
            transforms.Resize([height, width], interpolation=transforms.InterpolationMode.BILINEAR),
            transforms.CenterCrop([height, width]),
            transforms.ToTensor(),
        ]
    )


conditioning_image_pil = validation_image.resize([w_mask, h_mask])
conditioning_pixel_values = torch.stack([conditioning_image_transforms(conditioning_image_pil)])
conditioning_pixel_values = conditioning_pixel_values.to(memory_format=torch.contiguous_format).float()
print(conditioning_pixel_values.shape)
controlnet_image = conditioning_pixel_values.to(torch_device)
print(controlnet_image.shape)
print(conditioning_image_pil.mode)
conditioning_image_pil_l = conditioning_image_pil.convert('L')
print(conditioning_image_pil.mode)
print(conditioning_image_pil_l.mode)
conditioning_image_pil

In [ ]:
## crack_mask和原图组合
# new_crack_mask_np = np.zeros_like(label_np)

mask_crack_pil_zero = Image.fromarray(new_crack_mask_np)
mask_crack_pil_zero.paste(conditioning_image_pil_l, (xy_list[0][0], xy_list[1][1]))
mask_crack_pil_zero_rgb = mask_crack_pil_zero.convert('RGB')
mask_crack_rgb_pt = transforms.ToTensor()(mask_crack_pil_zero_rgb)
print(mask_crack_rgb_pt.shape)
mask_crack_pil_zero_rgb

### 组合init_image 和 选中的mask

In [ ]:
init_image_pt = transforms.ToTensor()(image_org)
print(init_image_pt.shape)
init_image_pt_4dim = init_image_pt.unsqueeze(0)   #####################  init_image_pt_4dim  #########################
print(init_image_pt_4dim.shape)
mask_pt = transforms.ToTensor()(mask_one_pil_zero_rgb)
print(mask_pt.shape)
masked_image = init_image_pt * (mask_pt < 0.5)
print(masked_image.shape)
transforms.ToPILImage()(masked_image)

### 组合init_image和crack_mask

In [ ]:
masked_crack_image = init_image_pt * (mask_crack_rgb_pt<0.01)
print(masked_crack_image.shape)
transforms.ToPILImage()(masked_crack_image)

### 组合init_image和crack_mask_all

In [ ]:
control_image = image_org.copy()
control_image.mode

In [ ]:

control_image.paste(conditioning_image_pil, (xy_list[0][0], xy_list[1][1]))
control_image_pt = transforms.ToTensor()(control_image)
print(control_image_pt.shape)           
control_image_pt_4_dim = control_image_pt.unsqueeze(0) ###############  control_image_pt_4_dim  #######################
control_image

## 1.创建各个部分的模型

In [ ]:

# control_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_crack_only_generate/checkpoint-2000/controlnet'
control_dir = '/home/ubunto/Project/konglx/generate/diffusers/examples/controlnet/controlnet-model_inpainting_h-256_w-256_2024-07-05_09:14:29_seeds-2024/checkpoint-2000/controlnet'
## 使用原来最初的config:
# config_dir = '/home/ubunto/Project/konglx/generate/ControlNet/models/stable-diffusion-v1-5'
## 使用inpainting的config:
config_dir = '/home/ubunto/Project/konglx/generate/diffusers/pretrained/stable-diffusion-inpainting'

In [ ]:
# from diffusers import AutoPipelineForInpainting

In [ ]:


vae = AutoencoderKL.from_pretrained(config_dir, subfolder='vae', use_safetensors=None)
tokenizer = CLIPTokenizer.from_pretrained(config_dir, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(config_dir, subfolder='text_encoder', use_safetensors=None)
# unet = UNet2DConditionModel.from_pretrained(config_dir, subfolder='unet', use_safetensors=None)
unet = UNet2DConditionModel.from_pretrained(config_dir, subfolder='unet', use_safetensors=None, 
                                            ignore_mismatched_sizes=True, low_cpu_mem_usage=False)
## ignore_mismatched_sizes=True, low_cpu_mem_usage=False ：这两个在in_channels与默认的4不同时，需要加上
controlnet = ControlNetModel.from_pretrained(control_dir)
controlnet.config

In [ ]:
from diffusers import UniPCMultistepScheduler

scheduler_multistep = UniPCMultistepScheduler.from_pretrained(config_dir, subfolder="scheduler")
# scheduler_multistep.config

In [ ]:
from diffusers import DDPMScheduler
scheduler_ddpm = DDPMScheduler.from_pretrained(config_dir, subfolder='scheduler')

In [ ]:
torch_device = "cuda"
vae.to(torch_device)
text_encoder.to(torch_device)
unet.to(torch_device)
controlnet.to(torch_device)
print('')
# unet.config
unet.config.in_channels

## 2.Create embeddings

### 2.1 Create text embeddings

In [ ]:
text_input = tokenizer(
    prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt"
)

with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

prompt_embeds = text_embeddings
encoder_hidden_states_control = text_embeddings
text_embeddings.shape

In [ ]:
'''
You’ll also need to generate the unconditional text embeddings which are the embeddings for the padding token.
These need to have the same shape (batch_size and seq_length) as the conditional text_embeddings:
'''
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer([""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt")
uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
uncond_embeddings.shape, text_embeddings.shape

In [ ]:
# Let’s concatenate the conditional and unconditional embeddings into a batch to avoid doing two forward passes:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
# text_embeddings
text_embeddings.shape, text_embeddings.dtype

In [ ]:
text_embeddings.shape

### 2.2 Create image embeddings

#### 2.2.1 control_image的embeddings

In [ ]:
import numpy as np
from typing import List, Optional, Tuple, Union
import PIL

def pil_to_numpy(images: Union[List[PIL.Image.Image], PIL.Image.Image]) -> np.ndarray:
    """
    Convert a PIL image or a list of PIL images to NumPy arrays.
    """
    if not isinstance(images, list):
        images = [images]
    images = [np.array(image).astype(np.float32) / 255.0 for image in images]
    images = np.stack(images, axis=0)

    return images

def numpy_to_pt(images: np.ndarray) -> torch.Tensor:
    """
    Convert a NumPy image to a PyTorch tensor.
    """
    if images.ndim == 3:
        images = images[..., None]

    images = torch.from_numpy(images.transpose(0, 3, 1, 2))
    return images

def pt_to_numpy(images: torch.Tensor) -> np.ndarray:
    """
    Convert a PyTorch tensor to a NumPy image.
    """
    images = images.cpu().permute(0, 2, 3, 1).float().numpy()
    return images

def img_normalize(images: torch.Tensor) -> torch.Tensor: # [-1, 1]
    return 2*images - 1

In [ ]:
# image_dir = '/home/ubunto/Project/konglx/generate/diffusers/datasets/corrosion_and_crack/corrosion_and_crack_dataset/target/11247-12.jpg'
# org_image = Image.open(image_dir)
# org_image_pt = transforms.ToTensor()(org_image)
# print(org_image_pt.shape, org_image_pt.min(), org_image_pt.max())

# org_image_norm_pt = img_normalize(org_image_pt)
# print(org_image_norm_pt.shape, org_image_norm_pt.min(), org_image_norm_pt.max())
# org_image

In [ ]:
print(conditioning_image_pil_l.mode)
conditioning_image_pil_l

In [ ]:
# # 如文献:https://onlinelibrary.wiley.com/doi/epdf/10.1111/mice.13171 的相加
# x0 = org_image_norm_pt # [3, 512, 512]
# mask_crack = transforms.ToTensor()(conditioning_image_pil_l)
# mask_crack_norm = img_normalize(mask_crack)
# # add_x0_
# # mask_crack.min(), mask_crack.max()
# # add_x0_mask = x0 + mask_crack
# print(x0.min(), x0.max(), x0.mean(), x0.std())
# print(mask_crack.min(), mask_crack.max(), mask_crack.mean(), mask_crack.std())
# print(mask_crack_norm.min(), mask_crack_norm.max(), mask_crack_norm.mean(), mask_crack_norm.std())

# add_x0_mask = x0 + mask_crack
# # add_x0_mask = x0 + mask_crack_norm
# add_norm = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(add_x0_mask)
# print(add_x0_mask.shape, add_norm.min(), add_norm.max(), add_norm.mean(), add_norm.std())
# transforms.ToPILImage()(add_x0_mask)


#### 2.2.2. masked_image的embeddings(暂不考虑)

In [ ]:
init_image_pt = transforms.ToTensor()(image_org)
print(init_image_pt.shape)
mask_pt = transforms.ToTensor()(mask_one_pil_zero_rgb)
print(mask_pt.shape)
masked_image_pt = init_image_pt * (mask_pt < 0.5)
print(masked_image_pt.shape)
transforms.ToPILImage()(masked_image_pt)   

In [ ]:
# prepare control image
def make_inpaint_condition(init_image, mask_image):
    init_image = np.array(init_image.convert("RGB")).astype(np.float32) / 255.0
    mask_image = np.array(mask_image.convert("L")).astype(np.float32) / 255.0

    assert init_image.shape[0:1] == mask_image.shape[0:1], "image and image_mask must have the same image size"
    init_image[mask_image > 0.5] = -1.0  # set as masked pixel
    init_image = np.expand_dims(init_image, 0).transpose(0, 3, 1, 2)
    init_image = torch.from_numpy(init_image)
    return init_image

In [ ]:
mask_inpaint_pt = make_inpaint_condition(init_image=image_org, mask_image=mask_one_pil_zero_rgb)
print(mask_inpaint_pt.shape)
transforms.ToPILImage()(mask_inpaint_pt.squeeze(0))

In [ ]:
# Copied from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.retrieve_latents
def retrieve_latents(
    encoder_output: torch.Tensor, generator: Optional[torch.Generator] = None, sample_mode: str = "sample"
):
    if hasattr(encoder_output, "latent_dist") and sample_mode == "sample":
        return encoder_output.latent_dist.sample(generator)
    elif hasattr(encoder_output, "latent_dist") and sample_mode == "argmax":
        return encoder_output.latent_dist.mode()
    elif hasattr(encoder_output, "latents"):
        return encoder_output.latents
    else:
        raise AttributeError("Could not access latents of provided encoder_output")

In [ ]:
# masked_image_latents = retrieve_latents(vae.encode(masked_image_pt.to(device=device)), generator=generator)
# masked_image_latents.shape

## 3.Create random noise (如果是*无inpainting*，dim=1的shape为unet.config.in_channels=4;如果是*有inpainting*，dim=1的shape=vae.config._in_channels=3)

In [ ]:
vae.config

### 3.1 *有inpainting*，dim=1的shape=vae.config._in_channels=3

In [ ]:
vae.config.scaling_factor

In [ ]:
# Copied from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline._encode_vae_image
def _encode_vae_image(image: torch.Tensor, generator: torch.Generator, vae=vae):
    if isinstance(generator, list):
        image_latents = [
            retrieve_latents(vae.encode(image[i : i + 1]), generator=generator[i])
            for i in range(image.shape[0])
        ]
        image_latents = torch.cat(image_latents, dim=0)
    else:
        image_latents = retrieve_latents(vae.encode(image), generator=generator)

    image_latents = vae.config.scaling_factor * image_latents

    return image_latents

In [ ]:
 # Copied from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline.prepare_mask_latents
def prepare_mask_latents(
    mask, masked_image, height, width, dtype, device, generator, encode_vae_image, batch_size=1, do_classifier_free_guidance=False
):
    # resize the mask to latents shape as we concatenate the mask to the latents
    # we do that before converting to dtype to avoid breaking in case we're using cpu_offload
    # and half precision
    mask = torch.nn.functional.interpolate(
        mask, size=(height // 8, width // 8)
    )
    mask = mask.to(device=device, dtype=dtype)

    masked_image = masked_image.to(device=device, dtype=dtype)

    if masked_image.shape[1] == 4:
        masked_image_latents = masked_image
    else:
        masked_image_latents = encode_vae_image(masked_image, generator=generator)

    # duplicate mask and masked_image_latents for each generation per prompt, using mps friendly method
    if mask.shape[0] < batch_size:
        if not batch_size % mask.shape[0] == 0:
            raise ValueError(
                "The passed mask and the required batch size don't match. Masks are supposed to be duplicated to"
                f" a total batch size of {batch_size}, but {mask.shape[0]} masks were passed. Make sure the number"
                " of masks that you pass is divisible by the total requested batch size."
            )
        mask = mask.repeat(batch_size // mask.shape[0], 1, 1, 1)
    if masked_image_latents.shape[0] < batch_size:
        if not batch_size % masked_image_latents.shape[0] == 0:
            raise ValueError(
                "The passed images and the required batch size don't match. Images are supposed to be duplicated"
                f" to a total batch size of {batch_size}, but {masked_image_latents.shape[0]} images were passed."
                " Make sure the number of images that you pass is divisible by the total requested batch size."
            )
        masked_image_latents = masked_image_latents.repeat(batch_size // masked_image_latents.shape[0], 1, 1, 1)

    mask = torch.cat([mask] * 2) if do_classifier_free_guidance else mask
    masked_image_latents = (
        torch.cat([masked_image_latents] * 2) if do_classifier_free_guidance else masked_image_latents
    )

    # aligning device to prevent device errors when concating it with the latent model input
    masked_image_latents = masked_image_latents.to(device=device, dtype=dtype)
    return mask, masked_image_latents

In [ ]:
mask_latent, masked_image_latent = prepare_mask_latents(mask=mask_one_pt_l_4dim, masked_image=control_image_pt_4_dim, 
                                                        height=mask_only_mask_pt_4dim.shape[2], width=mask_only_mask_pt_4dim.shape[3], 
                                                        dtype=text_embeddings.dtype, device=device, generator=generator, encode_vae_image=_encode_vae_image)


In [ ]:
mask_latent.shape, masked_image_latent.shape

In [ ]:
latents_vae = torch.randn(
    (batch_size, vae.config.in_channels, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)



latents_vae.shape, vae.config.in_channels

### 3.2. *无inpainting*，dim=1的shape为unet.config.in_channels=4

In [ ]:
''' 
Next, generate some initial random noise as a starting point for the diffusion process. 
This is the latent representation of the image, and it’ll be gradually denoised. 
At this point, the latent image is smaller than the final image size but that’s okay though 
because the model will transform it into the final 512x512 image dimensions later.
'''

# The height and width are divided by 8 because the vae model has 3 down-sampling layers.
# You can check by running the following:   2 ** (len(vae.config.block_out_channels) - 1) == 8

# do_classifier_free_guidance = False
# guess_mode = False

latents = torch.randn(
    (batch_size, unet.config.in_channels, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)



latents.shape

In [ ]:
# down_block_res_samples, mid_block_res_sample = controlnet(
#                     latents,
#                     2,
#                     encoder_hidden_states=encoder_hidden_states_control,
#                     controlnet_cond=controlnet_image,
#                     return_dict=False,
#                 )
# len(down_block_res_samples), down_block_res_samples[-1].shape, len(mid_block_res_sample), mid_block_res_sample[0].shape

## 4.Denoise the image

In [ ]:
'''
Start by scaling the input with the initial noise distribution, sigma, the noise scale value, 
which is required for improved schedulers like UniPCMultistepScheduler: 
'''
print(scheduler_multistep.init_noise_sigma)
latents = latents * scheduler_multistep.init_noise_sigma
latents.shape

In [ ]:
import PIL.Image
import numpy as np

def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)

In [ ]:
def show_latents(latents, vae):
    # scale and decode the image latents with vae
    latents_scaled = 1 / vae.config.scaling_factor * latents
    # pil_latents_scaled = transforms.ToPILImage()(latents.squeeze(0))
    # pil_latents_scaled = transforms.ToPILImage()(latents_scaled.squeeze(0))
    with torch.no_grad():
        # image = vae.decode(latents).sample
        image = vae.decode(latents_scaled).sample
    image = (image / 2 + 0.5).clamp(0, 1).squeeze()
    image = (image.permute(1, 2, 0) * 255).to(torch.uint8).cpu().numpy()
    image = Image.fromarray(image)
    return image


In [ ]:
encoder_hidden_states_control.shape, text_embeddings.shape

In [ ]:
'''  ***controlnet***
The last step is to create the denoising loop that’ll progressively 
transform the pure noise in latents to an image described by your prompt.
Remember, the denoising loop needs to do three things:

1.Set the scheduler’s timesteps to use during denoising.
2.Iterate over the timesteps.
3.At each timestep, call the UNet model to predict the noise residual and 
pass it to the scheduler to compute the previous noisy sample.

'''

from tqdm.auto import tqdm

def denoise_control_sd_v15(latents, num_inference_steps=25, scheduler = scheduler_multistep, controlnet = controlnet,
                           unet = unet, text_embeddings=text_embeddings, guidance_scale=5,
                           controlnet_image=controlnet_image, encoder_hidden_states=encoder_hidden_states_control,
                           weight_dtype = torch.float32):
    scheduler.set_timesteps(num_inference_steps)

  

    for i, t in enumerate(tqdm(scheduler.timesteps)):
        # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
        # if 
        latent_model_input = torch.cat([latents] * 2)
        # latent_model_input = latents
        latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)

        # controlnet
        down_block_res_samples, mid_block_res_sample = controlnet(
                        latents,
                        t,
                        encoder_hidden_states=encoder_hidden_states,  # encoder_hidden_states_control [1,77,768]
                        controlnet_cond=controlnet_image,
                        return_dict=False,
                    )
        
        
        # predict the noise residual
        with torch.no_grad():
            noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings, # text_embedding [2,77,768]
                                down_block_additional_residuals=[sample.to(dtype=weight_dtype) for sample in down_block_res_samples],
                                mid_block_additional_residual=mid_block_res_sample.to(dtype=weight_dtype),
                                return_dict=False,
                                )[0]

        # perform guidance
        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)



        # compute the previous noisy sample x_t -> x_t-1
        latents = scheduler.step(noise_pred, t, latents).prev_sample
        
        if (i + 1) % 5 == 0:
            display_sample(latents, i + 1)
            decoded_image_pil = show_latents(latents=latents, vae=vae)
            display(decoded_image_pil)
        
    return latents

In [ ]:
'''  ***controlnet_inpainting***
The last step is to create the denoising loop that’ll progressively 
transform the pure noise in latents to an image described by your prompt.
Remember, the denoising loop needs to do three things:

1.Set the scheduler’s timesteps to use during denoising.
2.Iterate over the timesteps.
3.At each timestep, call the UNet model to predict the noise residual and 
pass it to the scheduler to compute the previous noisy sample.

'''

from tqdm.auto import tqdm

def denoise_control_sd_v15_inpainting(latents, mask, masked_image_latents, num_inference_steps=25, scheduler = scheduler_multistep, controlnet = controlnet,
                           unet = unet, text_embeddings=text_embeddings, guidance_scale=5,
                           controlnet_image=controlnet_image, encoder_hidden_states=encoder_hidden_states_control,
                           weight_dtype = torch.float32, do_classifier_free_guidance=False):
    scheduler.set_timesteps(num_inference_steps)

  

    for i, t in enumerate(tqdm(scheduler.timesteps)):
        # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
        # if 
        latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
        # latent_model_input = latents
        latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)

        # controlnet
        down_block_res_samples, mid_block_res_sample = controlnet(
                        latents,
                        t,
                        encoder_hidden_states=encoder_hidden_states,  # encoder_hidden_states_control [1,77,768]
                        controlnet_cond=controlnet_image,
                        return_dict=False,
                    )
            # predict the noise residual
    # if num_channels_unet == 9:
        latent_model_input = torch.cat([latent_model_input, mask, masked_image_latents], dim=1)
        
        # predict the noise residual
        with torch.no_grad():
            noise_pred = unet(latent_model_input, t, encoder_hidden_states=encoder_hidden_states, # text_embedding [2,77,768]
                                down_block_additional_residuals=[sample.to(dtype=weight_dtype) for sample in down_block_res_samples],
                                mid_block_additional_residual=mid_block_res_sample.to(dtype=weight_dtype),
                                return_dict=False,
                                )[0]

        # # perform guidance
        # noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        # noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)



        # compute the previous noisy sample x_t -> x_t-1
        latents = scheduler.step(noise_pred, t, latents).prev_sample
        
        if (i + 1) % 5 == 0:
            display_sample(latents, i + 1)
            decoded_image_pil = show_latents(latents=latents, vae=vae)
            display(decoded_image_pil)
        
    return latents

In [ ]:
seed = 0
device = 'cuda'
generator = torch.Generator(device=device).manual_seed(seed) 
guidance_scale = 10


noise_randn =  torch.randn(
    (batch_size, 4, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)

noise_randn = noise_randn * scheduler_multistep.init_noise_sigma
print(noise_randn.shape)

latents_controled = denoise_control_sd_v15_inpainting(scheduler=scheduler_multistep, 
                                                      mask=mask_latent, masked_image_latents=masked_image_latent,
                                           latents=noise_randn,controlnet=controlnet, 
                                           controlnet_image=controlnet_image, 
                                           guidance_scale=guidance_scale,encoder_hidden_states=encoder_hidden_states_control,
                                           text_embeddings=text_embeddings)

In [ ]:
seed = 0
device = 'cuda'
generator = torch.Generator(device=device).manual_seed(seed) 
guidance_scale = 10


noise_randn =  torch.randn(
    (batch_size, 4, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)

noise_randn = noise_randn * scheduler_multistep.init_noise_sigma
print(noise_randn.shape)

latents_controled = denoise_control_sd_v15(scheduler=scheduler_multistep, 
                                           latents=noise_randn,controlnet=controlnet, 
                                           controlnet_image=controlnet_image, 
                                           guidance_scale=guidance_scale,encoder_hidden_states=encoder_hidden_states_control,
                                           text_embeddings=text_embeddings)

In [ ]:
seed = 0
device = 'cuda'
generator = torch.Generator(device=device).manual_seed(seed) 
guidance_scale = 10


noise_randn =  torch.randn(
    (batch_size, unet.config.in_channels, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)

noise_randn = noise_randn * scheduler_multistep.init_noise_sigma
print(noise_randn.shape)

latents_controled = denoise_control_sd_v15(scheduler=scheduler_multistep, 
                                           latents=noise_randn,controlnet=controlnet, 
                                           controlnet_image=controlnet_image, 
                                           guidance_scale=guidance_scale,encoder_hidden_states=encoder_hidden_states_control,
                                           text_embeddings=text_embeddings)


# show_latents(latents=latents_controled, vae=vae)

In [ ]:
seed = 0
device = 'cuda'
generator = torch.Generator(device=device).manual_seed(seed) 
guidance_scale = 10


noise_randn =  torch.randn(
    (batch_size, unet.config.in_channels, height // 8, width // 8),
    generator=generator,
    device=torch_device,
)

noise_randn = noise_randn * scheduler_multistep.init_noise_sigma
print(noise_randn.shape)

latents_controled = denoise_control_sd_v15(scheduler=scheduler_multistep, 
                                           latents=noise_randn,controlnet=controlnet, 
                                           controlnet_image=controlnet_image, 
                                           guidance_scale=guidance_scale,encoder_hidden_states=encoder_hidden_states_control,
                                           text_embeddings=text_embeddings)


# show_latents(latents=latents_controled, vae=vae)